# ExampleValidator Test Run

## Set up

TFX requires apache-airflow and docker SDK.

In [ ]:
!pip install 'apache-airflow[gcp]' docker tfx

In this notebook, we use TFX version 0.13.0

In [ ]:
import tfx
tfx.version.__version__

TFX requires TensorFlow >= 1.13.1

In [3]:
import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

'1.13.1'

TFX supports Python 3.5 from version 0.13.0

In [4]:
import sys
sys.version

'3.5.2 (default, Nov 12 2018, 13:43:14) \n[GCC 5.4.0 20160609]'

## Download sample data

In [5]:
%%bash
# This enables you to run this notebook twice.
# There should not be train/eval files at ~/taxi/data, since TFX can handle only single file with version 0.13.0
if [ -e ~/taxi/data ]; then
    rm -rf ~/taxi/data
fi

# download taxi data
mkdir -p ~/taxi/data/simple
mkdir -p ~/taxi/serving_model/taxi_simple
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv -O ~/taxi/data/simple/data.csv

# download 
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/taxi_utils.py -O ~/taxi/taxi_utils.py

--2019-06-13 10:32:45--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922668 (1.8M) [text/plain]
Saving to: ‘/root/taxi/data/simple/data.csv’

     0K .......... .......... .......... .......... ..........  2% 4.47M 0s
    50K .......... .......... .......... .......... ..........  5% 8.02M 0s
   100K .......... .......... .......... .......... ..........  7% 19.4M 0s
   150K .......... .......... .......... .......... .......... 10% 15.7M 0s
   200K .......... .......... .......... .......... .......... 13% 13.4M 0s
   250K .......... .......... .......... .......... .......... 15% 18.9M 0s
   300K .......... .......... .......... .......... .......... 18% 17.3M 0s
   350K ........

## Import

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import logging
import os
from google.protobuf import json_format

from tfx.components.base.base_component import ComponentOutputs
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.airflow.airflow_runner import AirflowDAGRunner
from tfx.orchestration.pipeline import Pipeline
from tfx.orchestration.tfx_runner import TfxRunner
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import csv_input
from tfx.utils.channel import Channel
from tfx.utils import types

/usr/local/lib/python3.5/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


## configs

In [7]:
# This example assumes that the taxi data is stored in ~/taxi/data and the
# taxi utility function is in ~/taxi.  Feel free to customize this as needed.
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_data_root = os.path.join(_taxi_root, 'data/simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model/taxi_simple')

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata')
_log_root = os.path.join(_tfx_root, 'logs')

# Airflow-specific configs; these will be passed directly to airflow
_airflow_config = {
    'schedule_interval': None,
    'start_date': datetime.datetime(2019, 1, 1),
}

# Logging overrides
logger_overrides = {'log_root': _log_root, 'log_level': logging.INFO}

## Create ExampleGen

In [8]:
"""Implements the chicago taxi pipeline with TFX."""
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
train_config = example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=2)
eval_config = example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        train_config,
        eval_config
    ]))

# Create outputs
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root, 'train/')

eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root, 'eval/')

output_dict = {'examples': Channel(
    type_name='ExamplesPath',
    static_artifact_collection=[train_examples, eval_examples])}

example_outputs = ComponentOutputs(output_dict)

example_gen = CsvExampleGen(
    input_base=examples, # A Channel of 'ExternalPath' type, it contains path of data source.
    output_config=output_config,  # An example_gen_pb2.Output instance, it contains train-eval split ratio.
    outputs=example_outputs # dict from name to output channel, it will be stored example_gen.outputs
)

## Create StatisticsGen

In [9]:
# Create outputs
train_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='train')
train_statistics.uri = os.path.join(_data_root, 'train/stats/')

eval_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='eval')
eval_statistics.uri = os.path.join(_data_root, 'eval/stats/')

output_dict = {'output': Channel(
    type_name='ExampleStatisticsPath',
    static_artifact_collection=[train_statistics, eval_statistics])}

statistics_outputs = ComponentOutputs(output_dict)

statistics_gen = StatisticsGen(
    input_data=example_gen.outputs.examples, # A Channel of 'ExamplesPath' type, it is equal to example_outputs
    name='Statistics Generator', # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=statistics_outputs # dict from name to output channel, it will be stored statistics_gen.outputs
)

## Create SchemaGen

In [10]:
# Create outputs
train_schema_path = types.TfxType(type_name='SchemaPath', split='train')
train_schema_path.uri = os.path.join(_data_root, 'train/schema/')

# NOTE: SchemaGen.executor can handle JUST ONE SchemaPath.
# Two or more SchemaPaths will cause ValueError
# such as "ValueError: expected list length of one but got 2".
schema_outputs = ComponentOutputs({
    'output':Channel(
        type_name='SchemaPath',
        static_artifact_collection=[train_schema_path] 
    )
})

schema_gen = SchemaGen(
    stats=statistics_gen.outputs.output, # A Channel of 'ExampleStatisticsPath' type, it is equal to statistics_outputs
    name='Schema Generator',  # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=schema_outputs # dict from name to output channel, it will be stored schema_gen.outputs
)

## Create ExampleValidator

In [11]:
eval_validator_path = types.TfxType('ExampleValidationPath', split='eval')
eval_validator_path.uri = os.path.join(_data_root, 'eval/validation/')

# NOTE: ExampleValidator.executor can handle JUST ONE ExampleValidationPath
# since ExampleValidator should check only 'eval' data.
# Two or more ExampleValidationPath will cause ValueError
# such as "ValueError: expected list length of one but got 2".
validator_outputs = ComponentOutputs({
    'output':Channel(
        type_name='ExampleValidationPath',
        static_artifact_collection=[eval_validator_path]
    )
})

example_validator = ExampleValidator(
    stats=statistics_gen.outputs.output,
    schema=schema_gen.outputs.output,
    name='Example Validator',
    outputs=validator_outputs
)

## Create pipeline

In [12]:
pipeline = Pipeline(
    pipeline_name="TFX Pipeline",
    pipeline_root=_pipeline_root,
    components=[example_gen, statistics_gen, schema_gen, example_validator]
)

## Execute

In [13]:
class DirectRunner(TfxRunner):
    """Tfx runner on local"""
    
    def __init__(self, config=None):
        self._config = config or {}
    
    def run(self, pipeline):
        for component in pipeline.components:
            self._execute_component(component)
            
        return pipeline
            
    def _execute_component(self, component):
        input_dict = {key:value.get() for key, value in component.input_dict.items()}
        output_dict = {key: value.get() for key, value in component.outputs.get_all().items()}
        exec_properties = component.exec_properties
        executor = component.executor()
        executor.Do(input_dict, output_dict, exec_properties)

In [14]:
pipeline = DirectRunner().run(pipeline)

INFO:tensorflow:Starting Executor execution.
[2019-06-13 10:32:50,003] {base_executor.py:72} INFO - Starting Executor execution.
INFO:tensorflow:Inputs for Executor is: {"input-base": [{"artifact": {"uri": "/root/taxi/data/simple", "properties": {"type_name": {"stringValue": "ExternalPath"}, "split": {"stringValue": ""}}}, "artifact_type": {"name": "ExternalPath", "properties": {"span": "INT", "name": "STRING", "type_name": "STRING", "split": "STRING", "state": "STRING"}}}]}
[2019-06-13 10:32:50,007] {base_executor.py:74} INFO - Inputs for Executor is: {"input-base": [{"artifact": {"uri": "/root/taxi/data/simple", "properties": {"type_name": {"stringValue": "ExternalPath"}, "split": {"stringValue": ""}}}, "artifact_type": {"name": "ExternalPath", "properties": {"span": "INT", "name": "STRING", "type_name": "STRING", "split": "STRING", "state": "STRING"}}}]}
INFO:tensorflow:Outputs for Executor is: {"examples": [{"artifact": {"uri": "/root/taxi/data/simple/train/", "properties": {"type_

[2019-06-13 10:32:57,800] {fn_api_runner.py:437} INFO - Running ((ref_PCollection_PCollection_24/Read)+(ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/PreFinalize_50))+(ref_PCollection_PCollection_33/Write)
[2019-06-13 10:32:57,822] {fn_api_runner.py:437} INFO - Running (ref_PCollection_PCollection_24/Read)+(ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/FinalizeWrite_51)
[2019-06-13 10:32:57,835] {filebasedsink.py:290} INFO - Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
[2019-06-13 10:32:57,941] {filebasedsink.py:327} INFO - Renamed 1 shards in 0.10 seconds.
[2019-06-13 10:32:57,964] {fn_api_runner.py:437} INFO - Running (((ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/DoOnce/Read_66)+(ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/InitializeWrite_67))+(ref_PCollection_PCollection_42/Write))+(ref_PCollection_PCollection_43/Write)
[2019-06-13 10:32:57,979] {fn_api_runner.py:437} INFO - Running ((ShuffleSplite

[2019-06-13 10:33:03,005] {fn_api_runner.py:437} INFO - Running ((((ref_AppliedPTransform_ReadData.train/Read_106)+(ref_AppliedPTransform_DecodeData.train/ParseTFExamples_108))+(((((ref_AppliedPTransform_GenerateStatistics.train/RunStatsGenerators/KeyWithVoid_111)+((ref_AppliedPTransform_GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/ParDo(SplitHotCold)/ParDo(SplitHotCold)_115)+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/Flatten/Transcode/0)))+(ref_AppliedPTransform_GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/TopKUniques_ConvertInputToFeatureValuesWithWeights_135))+(((ref_AppliedPTransform_GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/WindowIntoDiscarding_116)+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PreCombineFn)/Precombine)

[2019-06-13 10:33:09,597] {fn_api_runner.py:437} INFO - Running (((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/MergeDatasetFeatureStatisticsProtos/Group/Read)+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/MergeDatasetFeatureStatisticsProtos/Merge))+((((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/MergeDatasetFeatureStatisticsProtos/ExtractOutputs)+(ref_AppliedPTransform_GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/AddSliceKeyToStatsProto_177))+(ref_AppliedPTransform_GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/ToList/ToList/KeyWithVoid_180))+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/ToList/ToList/CombinePerKey/Precombine)))+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/ToList/ToList/CombinePerKey/Group/Write)
[2019-06-13 10:33:09,636] {fn_api_runner.py:437} INFO - Running (GenerateStatist

[2019-06-13 10:33:13,656] {fn_api_runner.py:437} INFO - Running (GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/Uniques_CountPerFeatureName/CombinePerKey(CountCombineFn)/Group/Read)+((((GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/Uniques_CountPerFeatureName/CombinePerKey(CountCombineFn)/Merge)+(GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/Uniques_CountPerFeatureName/CombinePerKey(CountCombineFn)/ExtractOutputs))+(ref_AppliedPTransform_GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/Uniques_ConvertToSingleFeatureStats_63))+(GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/FlattenTopKUniquesResults/Write/1))
[2019-06-13 10:33:13,680] {fn_api_runner.py:437} INFO - Running ((GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatis

[2019-06-13 10:33:14,255] {base_executor.py:74} INFO - Inputs for Executor is: {"schema": [{"artifact": {"uri": "/root/taxi/data/simple/train/schema/", "properties": {"type_name": {"stringValue": "SchemaPath"}, "split": {"stringValue": "train"}}}, "artifact_type": {"name": "SchemaPath", "properties": {"span": "INT", "name": "STRING", "type_name": "STRING", "split": "STRING", "state": "STRING"}}}], "stats": [{"artifact": {"uri": "/root/taxi/data/simple/train/stats/", "properties": {"type_name": {"stringValue": "ExampleStatisticsPath"}, "split": {"stringValue": "train"}}}, "artifact_type": {"name": "ExampleStatisticsPath", "properties": {"span": "INT", "name": "STRING", "type_name": "STRING", "split": "STRING", "state": "STRING"}}}, {"artifact": {"uri": "/root/taxi/data/simple/eval/stats/", "properties": {"type_name": {"stringValue": "ExampleStatisticsPath"}, "split": {"stringValue": "eval"}}}, "artifact_type": {"name": "ExampleStatisticsPath", "properties": {"span": "INT", "name": "STRI

## Check Result

In [15]:
!ls -Rlhs /root/taxi/data/simple/

/root/taxi/data/simple/:
total 1.9M
1.9M -rw-r--r-- 1 root root 1.9M Jun 13 10:32 data.csv
4.0K drwxr-xr-x 4 root root 4.0K Jun 13 10:33 eval
4.0K drwxr-xr-x 4 root root 4.0K Jun 13 10:33 train

/root/taxi/data/simple/eval:
total 212K
204K -rw-r--r-- 1 root root 201K Jun 13 10:32 data_tfrecord-00000-of-00001.gz
4.0K drwxr-xr-x 2 root root 4.0K Jun 13 10:33 stats
4.0K drwxr-xr-x 2 root root 4.0K Jun 13 10:33 validation

/root/taxi/data/simple/eval/stats:
total 20K
20K -rw-r--r-- 1 root root 17K Jun 13 10:33 stats_tfrecord

/root/taxi/data/simple/eval/validation:
total 8.0K
8.0K -rw-r--r-- 1 root root 6.1K Jun 13 10:33 anomalies.pbtxt

/root/taxi/data/simple/train:
total 416K
408K -rw-r--r-- 1 root root 406K Jun 13 10:32 data_tfrecord-00000-of-00001.gz
4.0K drwxr-xr-x 2 root root 4.0K Jun 13 10:33 schema
4.0K drwxr-xr-x 2 root root 4.0K Jun 13 10:33 stats

/root/taxi/data/simple/train/schema:
total 8.0K
8.0K -rw-r--r-- 1 root root 4.5K Jun 13 10:33 schema.pbtxt

/root/taxi/data/simple/tr

In [16]:
!cat /root/taxi/data/simple/eval/validation/anomalies.pbtxt

baseline {
  feature {
    name: "trip_seconds"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "pickup_latitude"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "pickup_longitude"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "trip_start_hour"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "company"
    value_count {
      min: 1
      max: 1
    }
    type: BYTES
    domain: "company"
    presence {
      min_count: 1
    }
  }
  feature {
    name: "pickup_community_area"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      mi

`anomalies.pbtxt` is only simple text file. To visualize it, we are going to;

1. get the path of `anomalies.pbtxt`from `example_validator`
2. parse `anomalies.pbtxt` into anomalies (protobuf)
3. visualize schema with [tfdv](https://www.tensorflow.org/tfx/data_validation/get_started)


In [28]:
# 1. get the path of `schema.pbtxt`
def get_validation_directory(example_validator):
    output_dict = {key: value.get() for key, value in example_validator.outputs.get_all().items()}
    directory = types.get_split_uri(output_dict['output'], 'eval')
    return directory

validation_directory = get_validation_directory(example_validator)

In [29]:
# 2. parse schema.pbtxt

from pathlib import Path
from google.protobuf.text_format import Parse
from google.protobuf.message import Message
from google.protobuf import text_format
from tensorflow_metadata.proto.v0 import anomalies_pb2

def parse_anomalies_proto_string(validation_directory):
    path = Path(validation_directory)
    for filepath in path.glob('*'):
        with open(str(filepath), 'r') as file: # since we are using python 3.5, not 3.6+
            anomalies_proto_string = file.read()
            anomalies = anomalies_pb2.Anomalies()
            text_format.Parse(anomalies_proto_string, anomalies)
            return anomalies

anomalies = parse_anomalies_proto_string(validation_directory)

In [31]:
# 3. visualize anomalies with tfdv

import tensorflow_data_validation as tfdv
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'company',Unexpected string values,"Examples contain values missing from the schema: 1085 - 72312 N and W Cab Co (<1%), 2192 - Zeymane Corp (<1%), 2809 - 95474 C & D Cab Co Inc. (<1%), 3897 - 57856 Ilie Malec (<1%), 4615 - Tyrone Henderson (<1%), 585 - Valley Cab Co (<1%), 5874 - Sergey Cab Corp. (<1%), 5997 - AW Services Inc. (<1%), 6057 - 24657 Richard Addo (<1%), 6574 - Babylon Express Inc. (<1%), 6743 - Luhak Corp (<1%)."
